In [1]:
from mkndaq.inst.neph import NEPH

cfg = {'ne300': {
                'type': 'NE300',
                'serial_number': '23-0690',
                'serial_id': 0,
                'protocol': 'acoem',
                'socket': {
                    'host': '192.168.0.50',
                    'port': 32783,
                    'timeout': 0.1,
                    # 'sleep': 0.1,
                },
                # 'set_datetime': '',
                # 'sampling_interval': '1',
                'staging_zip': True,  
                'verbosity': 2,  # 0: silent, 1: medium, 2: full          
            },
                'reporting_interval': '10',
                'logs': 'mkndaq/logs',
                'data': 'mkndaq/data',
                'staging': {
                    'path': 'mkndaq/staging',
                },
}

ne300 = NEPH('ne300', cfg)

# Initialize NE300 (name: ne300  S/N: 23-0690)


In [3]:
n = 3.2
f"{n:2.0f}"

' 3'

In [2]:
ne300.get_id()

msg: b'\x02\x00\x01\x03\x00\x00\x00\x04'
rcvd (raw): b'\x02\x00\x01\x03\x00\x10\x00\x00\x00\x9e\x00\x00\x01,\x00\x00\x00\x00\x00\x00\x00\x00\xa3\x04'
rcvd: b'\x02\x00\x01\x03\x00\x10\x00\x00\x00\x9e\x00\x00\x01,\x00\x00\x00\x00\x00\x00\x00\x00\xa3\x04'
response length: 4
6: 158
10: 300
14: 0
18: 0
msg: b'\x02\x00\x02\x03\x00\x00\x03\x04'
rcvd (raw): b'\x02\x00\x02\x03\x00\x08\x00\x00\x01,\x00\x00\x00\x00&\x04'
rcvd: b'\x02\x00\x02\x03\x00\x08\x00\x00\x01,\x00\x00\x00\x00&\x04'
response length: 2
6: 300
10: 0


[300, 0]

In [5]:
ne300.get_datetime()

msg: b'\x02\x00\x04\x03\x00\x04\x00\x00\x00\x01\x00\x04'
rcvd (raw): b'\x02\x00\x04\x03\x00\x04\x85d8e\xbd\x04'
rcvd: b'\x02\x00\x04\x03\x00\x04\x85d8e\xbd\x04'
response length: 1
6: 2237937765
0b10000101011001000011100001100101


datetime.datetime(2033, 5, 18, 3, 33, 37)

In [6]:
import time
dtm = time.localtime(2000000000)
dtm = time.gmtime()
ne300.set_datetime(dtm=dtm)
ne300.get_datetime()

time.struct_time(tm_year=2024, tm_mon=4, tm_mday=24, tm_hour=15, tm_min=26, tm_sec=41, tm_wday=2, tm_yday=115, tm_isdst=0)
msg: b'\x02\x00\x05\x03\x00\x08\x00\x00\x00\x01a0\xf6\xa9\x03\x04'
rcvd (raw): b''
rcvd: b''
response length: 0
msg: b'\x02\x00\x04\x03\x00\x04\x00\x00\x00\x01\x00\x04'
rcvd (raw): b'\x02\x00\x04\x03\x00\x04a0\xf6\xa9\x0f\x04'
rcvd: b'\x02\x00\x04\x03\x00\x04a0\xf6\xa9\x0f\x04'
response length: 1
6: 1630598825
0b1100001001100001111011010101001


datetime.datetime(2024, 4, 24, 15, 26, 41)

In [ ]:
import datetime
import time

dtm = time.gmtime(2000000000)
SS = bin(dtm.tm_sec)[2:].zfill(6)
MM = bin(dtm.tm_min)[2:].zfill(6)
HH = bin(dtm.tm_hour)[2:].zfill(5)
dd = bin(dtm.tm_mday)[2:].zfill(5)
mm = bin(dtm.tm_mon)[2:].zfill(4)
yyyy = bin(dtm.tm_year - 2000).zfill(6)
ab = (yyyy + mm + dd + HH + MM + SS)
setdtm = (int(yyyy + mm + dd + HH + MM + SS, base=2)).to_bytes(4)
bytes([int(ab[2:10], base=2), int(ab[10:18], base=2), int(ab[18:26], base=2), int(ab[26:34], base=2)])


In [ ]:
# import socket
# import time

# def tcp(msg) -> bytes:
#     with socket.socket(socket.AF_INET, socket.SOCK_STREAM, ) as s:
#         # connect to the server
#         s.settimeout(0.1)
#         s.connect(("192.168.0.50", 32783))
#         s.sendall(msg)
#         print(f"msg: {msg}")

#         # receive response
#         rcvd = b""
#         while True:
#             try:
#                 data = s.recv(1024)
#                 rcvd = rcvd + data
#                 if chr(4).encode() in rcvd:
#                     break
#             except:
#                 break
#         print(f"rcvd (raw): {rcvd}")
#         rcvd = rcvd.replace(b"\xff\xfb\x01\xff\xfe\x01\xff\xfb\x03", b"")
#         print(f"rcvd: {rcvd}")

#         msg_len = int(int.from_bytes(rcvd[4:6]) / 4)
#         print(f"msg length: {msg_len}")
        
#         result = []
#         for i in range(6, (msg_len + 1) * 4 + 2, 4):
#             token = int.from_bytes(rcvd[i:(i+4)])
#             print(f"{i}: {token}")
#             result.append(token)
#     return result

In [ ]:
# send data using ACOEM protocol
# Byte  |1  |2  |3  |4  |5..6   |7..10      |11 |12
#       |STX|sID|CMD|ETX|len    |data       |CS |EOT
# chr(2) = STX
# chr(3) = ETX
# chr(4) = EOT

def tcp2(serial_id: int, command: str, message_data: bytes) -> bytes:
    commands = {
        # 'get_error': bytes([0]),
        'get_instr_type': 1,
        'get_version': 2,
        'reset': 3,
        'get_values': 4,
        }
    msgl = len(message_data)
    if msgl==0:
        msg = bytes([2, serial_id, commands[command], 3, 0, msgl])
    else:
        msg = bytes([2, serial_id, commands[command], 3, 0, msgl]) + message_data
    checksum = chksum(msg)
    msg += checksum + bytes([4])

    with socket.socket(socket.AF_INET, socket.SOCK_STREAM, ) as s:
        # connect to the server
        s.settimeout(0.1)
        s.connect(("192.168.0.50", 32783))

        s.sendall(msg)
        print(f"msg: {msg}")

        # receive response
        rcvd = b""
        while True:
            try:
                data = s.recv(1024)
                rcvd = rcvd + data
                if chr(4).encode() in rcvd:
                    break
            except:
                break
        print(f"rcvd (raw): {rcvd}")
        rcvd = rcvd.replace(b"\xff\xfb\x01\xff\xfe\x01\xff\xfb\x03", b"")
        print(f"rcvd: {rcvd}")

        msg_len = int(int.from_bytes(rcvd[4:6]) / 4)
        print(f"msg length: {msg_len}")
        
        result = []
        for i in range(6, (msg_len + 1) * 4 + 2, 4):
            token = int.from_bytes(rcvd[i:(i+4)])
            print(f"{i}: {token}")
            result.append(token)
    return result

In [ ]:
rcvd = tcp2(0, 'get_version', '')
rcvd = tcp2(0, 'get_instr_type', '')


In [ ]:
import datetime


In [ ]:
# read current datetime (Aurora Table 46 Parameter 1)
rcvd = tcp2(0, 'get_values', bytes([0,0,0,1]))
print(acoem_timestamp_to_datetime(token=rcvd))

In [ ]:
str = bytes([2,0,0,3,0,4])
print(chksum(str))

In [ ]:
print("read firmware version")
msg = bytes([2,0,2,3,0,0,3,4])
rcvd = tcp(msg)
print(rcvd)
# read firmware version
# sent: b'\x02\x00\x02\x03\x00\x00\x03\x04'
# rcvd: b'\x02\x00\x02\x03\x00\x08\x00\x00\x01,\x00\x00\x00\x00&\x04'
# msg length: 2
# 6: 300
# 10: 0

In [ ]:
print("read instrument type")
msg = bytes([2,0,1,3,0,0,0,4])
rcvd = tcp(msg)
print(rcvd)

In [ ]:
print("get value (primary gas data)")
msg = bytes([2,0,4,3,0,4,0,0,0,32,36,4])
rcvd = tcp(msg)
print(rcvd)

In [ ]:
print("read error")
msg = bytes([2,0,0,3,0,0,0,4])

print(f"sent: {msg}")
rcvd = tcp(msg)
print(f"rcvd: {rcvd}")
msg_len = int(int.from_bytes(rcvd[4:6]) / 4)
print(f"msg length: {msg_len}")
for i in range(6, (msg_len + 1) * 4 + 2, 4):
    print(f"{i}: {int.from_bytes(rcvd[i:(i+4)])}")

In [ ]:
ne300.do_zero_check()
time.sleep(60)
for i in range(10):
    ne300.get_data(save=True)
    time.sleep(10)

ne300.do_ambient()
for i in range(10):
    ne300.get_data(save=True)
    time.sleep(10)


In [ ]:
import time
for i in range(3):
    print(ne300.get_data(sep=",", save=False, get_status_word=False))
    time.sleep(10)

In [ ]:
# print(ne300.do_zero_check())
print(ne300.get_all_new_data(save=False, sep="|"))

# Aurora NE-300 v5.0 ID #230690

# (0, 'Aurora NE-300 v5.0 ID #230690\r\n')
# 2024-01-25 14:37:37 .get_data (name=ne300, save=False)
# (0, '28/2/2024 15:57:00|60|0|1.341209e+01|1.900581e+01|2.459902e+01|2.218892e+00|2.862914e+00|3.427358e+00|2.630001e+01|9.678900e+02|2.457000e+01|2.895999e+01|9.690800e+02|2.708000e+01|6.007494e+00|1.470621e-02|1.693243e-02|1.907027e-02|2.333371e+02|1.522294e+06|1.909421e+06|1.402254e+06|1.789572e+04|2.621301e+04|2.658940e+04|1.161944e+04|1.474019e+04|1.391521e+04|2.418457e+01|1.744695e+03|1.836182e+01|4.848633e+00|2.049459e+01|1.227465e-02|1.426282e-02|1.897962e-02|7.924382e-03|7.973331e-03|9.864524e-03|6.607600e+01|0.000000e+00|5.020050e+01|0.000000e+00|\r\n\n28/2/2024 15:57:00|60|0|1.341209e+01|1.900581e+01|2.459902e+01|2.218892e+00|2.862914e+00|3.427358e+00|2.630001e+01|9.678900e+02|2.457000e+01|2.895999e+01|9.690800e+02|2.708000e+01|6.007494e+00|1.470621e-02|1.693243e-02|1.907027e-02|2.333371e+02|1.522294e+06|1.909421e+06|1.402254e+06|1.789572e+04|2.621301e+04|2.658940e+04|1.161944e+04|1.474019e+04|1.391521e+04|2.418457e+01|1.744695e+03|1.836182e+01|4.848633e+00|2.049459e+01|1.227465e-02|1.426282e-02|1.')
# Instrument ne300 status: 30/11/2023 00:00:00,60,0,9.765964e-01,1.155139e+00,1.231401e+00,1.073153e-01,1.748719e-01,2.522535e-01,3.354000e+01,1.002410e+03,2.996000e+01,2.644000e+01,1.004090e+03,5.332000e+01,6.020748e+00,5.000000e-03,5.000000e-03,5.000000e-03,2.933394e+02,1.494557e+06,1.888788e+06,1.392557e+06,1.470679e+04,1.969712e+04,1.927430e+04,1.087828e+04,1.395863e+04,1.322223e+04,2.410400e+01,1.879666e+03,1.839111e+01,5.090332e+00,2.026616e+01,1.035651e-02,1.090555e-02,1.380057e-02,7.627005e-03,7.697072e-03,9.421869e-03,7.000000e+01,2.900000e+01,4.880183e+01,0.000000e+00,
# 30/11/2023 00:01:00,60,0,9.642181e-01,1.143413e+00,1.238636e+00,1.114373e-01,1.734347e-01,2.362185e-01,3.351000e+01,1.002670e+03,2.996000e+01,2.638000e+01,1.003850e+03,5.332000e+01,5.982811e+00,1.746927e-02,5.000000e-03,5.000000e-03,2.433375e+02,1.493905e+06,1.891611e+06,1.389815e+06,1.404880e+04,1.947819e+04,1.927932e+04,1.091333e+04,1.383672e+04,1.295173e+04,2.412598e+01,1.708967e+03,1.841309e+01,4.965820e+00,2.026616e+01,9.923033e-03,1.079363e-02,1.
# (0, '30/11/2023 00:00:00,60,0,9.765964e-01,1.155139e+00,1.231401e+00,1.073153e-01,1.748719e-01,2.522535e-01,3.354000e+01,1.002410e+03,2.996000e+01,2.644000e+01,1.004090e+03,5.332000e+01,6.020748e+00,5.000000e-03,5.000000e-03,5.000000e-03,2.933394e+02,1.494557e+06,1.888788e+06,1.392557e+06,1.470679e+04,1.969712e+04,1.927430e+04,1.087828e+04,1.395863e+04,1.322223e+04,2.410400e+01,1.879666e+03,1.839111e+01,5.090332e+00,2.026616e+01,1.035651e-02,1.090555e-02,1.380057e-02,7.627005e-03,7.697072e-03,9.421869e-03,7.000000e+01,2.900000e+01,4.880183e+01,0.000000e+00,\r\n30/11/2023 00:01:00,60,0,9.642181e-01,1.143413e+00,1.238636e+00,1.114373e-01,1.734347e-01,2.362185e-01,3.351000e+01,1.002670e+03,2.996000e+01,2.638000e+01,1.003850e+03,5.332000e+01,5.982811e+00,1.746927e-02,5.000000e-03,5.000000e-03,2.433375e+02,1.493905e+06,1.891611e+06,1.389815e+06,1.404880e+04,1.947819e+04,1.927932e+04,1.091333e+04,1.383672e+04,1.295173e+04,2.412598e+01,1.708967e+03,1.841309e+01,4.965820e+00,2.026616e+01,9.923033e-03,1.079363e-02,1')
# 2024-01-25 14:23:26 .get_data (name=ne300, save=False)
# (0, '28/2/2024 15:43:17,13.810564,2.414614,19.843826,2.683266,24.489584,3.266213,25.60,0,0.00,967.7,0,0\r\n')
# 2024-01-25 14:23:36 .get_data (name=ne300, save=False)
# (0, '28/2/2024 15:43:28,13.763916,2.429190,19.711246,2.699752,24.422705,3.274734,25.61,0,0.00,967.8,0,0\r\n')
# 2024-01-25 14:23:47 .get_data (name=ne300, save=False)
# (0, '28/2/2024 15:43:38,13.717136,2.425864,19.598263,2.699717,24.439505,3.250718,25.61,0,0.00,967.7,0,0\r\n')
# # Initialize NE300 (name: ne300  S/N: 23-0690)
# b'\x020x0'

In [ ]:
# stx = bytes(chr(2).encode())
# cmd = r'1'.encode()
# etx = chr(3).encode()
# zip(stx, cmd, etx)
# bytes([a ^ b ^ c for a, b, c in zip(stx, cmd, etx)])